In [ ]:
%cd drive/MyDrive/randomCNN-voice-transfer-master/

[Errno 2] No such file or directory: 'drive/MyDrive/randomCNN-voice-transfer-master/'
/content/drive/MyDrive/randomCNN-voice-transfer-master


In [ ]:
!pip install -r requirements.txt

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train.py -content input/boy18.wav -style input/girl52.wav

a_content:  (257, 244)
a_style:  (257, 355)
a_content_torch:  torch.Size([1, 1, 257, 244])
a_style_torch:  torch.Size([1, 1, 257, 355])
a_C_var:  torch.Size([1, 1, 257, 244])
a_S_var:  torch.Size([1, 1, 257, 355])
x_delta:  torch.Size([1, 1, 257, 244])
out:  torch.Size([1, 32, 255, 244])
x_delta:  torch.Size([1, 1, 257, 355])
out:  torch.Size([1, 32, 255, 355])
a_C:  torch.Size([1, 32, 255, 244])
a_S:  torch.Size([1, 32, 255, 355])
a_G_var:  torch.Size([1, 1, 257, 244])
x_delta:  torch.Size([1, 1, 257, 244])
out:  torch.Size([1, 32, 255, 244])
a_G: %d torch.Size([1, 32, 255, 244])
x_delta:  torch.Size([1, 1, 257, 244])
out:  torch.Size([1, 32, 255, 244])
a_G: %d torch.Size([1, 32, 255, 244])
x_delta:  torch.Size([1, 1, 257, 244])
out:  torch.Size([1, 32, 255, 244])
a_G: %d torch.Size([1, 32, 255, 244])
x_delta:  torch.Size([1, 1, 257, 244])
out:  torch.Size([1, 32, 255, 244])
a_G: %d torch.Size([1, 32, 255, 244])
x_delta:  torch.Size([1, 1, 257, 244])
out:  torch.Size([1, 32, 255, 244]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [37]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import random
import scipy
import imageio
from scipy.optimize import fmin_l_bfgs_b   # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_l_bfgs_b.html
import warnings
tf.compat.v1.disable_eager_execution()

import tensorflow as tf
from tensorflow import keras

import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

import librosa
import numpy as np

import time
import math
import argparse

random.seed(1618)
np.random.seed(1618)
#tf.set_random_seed(1618)   # Uncomment for TF1.
tf.random.set_seed(1618)

#tf.logging.set_verbosity(tf.logging.ERROR)   # Uncomment for TF1.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


# coefficient of content and style
style_param = 2
content_param = 1e-3

num_epochs = 20

N_FFT = 512
N_CHANNELS = round(1 + N_FFT/2)
OUT_CHANNELS = 32
N_FILTERS  = 4096

content = "drive/MyDrive/randomCNN-voice-transfer-master/input/boy18.wav"
style = "drive/MyDrive/randomCNN-voice-transfer-master/input/girl52.wav"


#=============================<Helper Fuctions>=================================

def preprocessing(content, style):
    a_content, sr = wav2spectrum(content)
    a_style, sr = wav2spectrum(style)
    N_SAMPLES = min(a_content.shape[1],a_style.shape[1])
    a_style = a_style[:N_CHANNELS, :N_SAMPLES]
    return a_content, a_style, N_SAMPLES, sr

def wav2spectrum(filename):
    x, sr = librosa.load(filename)
    S = librosa.stft(x, N_FFT)
    p = np.angle(S)
    S = np.log1p(np.abs(S))
    return S, sr

def spectrum2wav(spectrum, sr, outfile):
    a = np.exp(spectrum) - 1
    p = 2 * np.pi * np.random.random_sample(spectrum.shape) - np.pi
    for i in range(50):
        S = a * np.exp(1j * p)
        x = librosa.istft(S)
        p = np.angle(librosa.stft(x, N_FFT))
    librosa.output.write_wav(outfile, x, sr)

def plt_spectrum(content, name):
    plt.figure(figsize=(5, 5))
    plt.subplot(1, 1, 1)
    plt.imsave( name+'.png', content[:400, :])

#========================<Loss Function Builder Functions>======================
def gramMatrix(x):
    m, n_C, n_H, n_W = x.shape
    x_unrolled = tf.reshape(x,(m * n_C * n_H, n_W))
    g = tf.matmul(x_unrolled,tf.transpose(x_unrolled)) / 244
    return g
    
def compute_content_loss(a_C, a_G):
    content_loss = tf.nn.l2_loss(a_C - a_G)
    return content_loss

def compute_layer_style_loss(a_S, a_G):
    style_gram = gramMatrix(a_S)
    gram = gramMatrix(a_G)
    style_loss = 4 * tf.nn.l2_loss(gram - style_gram)
    return style_loss

def total_loss(a_C, a_S, a_G):
    content_loss = content_param * compute_content_loss(a_C, a_G)
    style_loss = style_param * compute_layer_style_loss(a_S, a_G)
    return content_loss + style_loss


#=========================<Pipeline Functions>==================================
class Evaluator(object):
    def __init__(self, kFunction):
        self.kFunction = kFunction
        
    def loss(self, x):
        self.loss_, self.grads_ = self.kFunction([x.reshape((1,1, N_CHANNELS, round(x.shape[0]/N_CHANNELS)))])
        return self.loss_.astype(np.float64)

    def grads(self, x):
        return self.grads_.flatten().astype(np.float64)

def CNN():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=N_FILTERS, kernel_size=1, strides=(1, 1), padding='VALID'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    return model

def styleTransfer(a_content, a_style, N_SAMPLES):
    print("   Building transfer model.")
    a_C = tf.convert_to_tensor(a_content)[None, None, :, :]
    a_S = tf.convert_to_tensor(a_style)[None, None, :, :]
    a_G = K.placeholder(a_C.shape)

    model = CNN()
    content_features = model(K.variable(a_C))
    style_features = model(K.variable(a_S))
    gen_features = model(a_G)

    # Get the loss
    loss = total_loss(content_features, style_features, gen_features)

    # Setup gradients or use K.gradients().
    grads = K.gradients(loss, a_G)
    kFunction = K.function([a_G], [loss] + grads)
    evaluator = Evaluator(kFunction)
    x = np.zeros((1,1, N_CHANNELS, N_SAMPLES))
    print("   Beginning transfer.")
    # Train the Model
    for epoch in range(1, num_epochs + 1):
        res = scipy.optimize.fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=500)
        x = res[0].reshape((1,1, N_CHANNELS, N_SAMPLES))
    return x
    

#=========================<Main>================================================

def main():
    print("Starting audio style transfer program.")
    a_content, a_style, N_SAMPLES, sr = preprocessing(content, style)
    x = styleTransfer(a_content, a_style, N_SAMPLES)
    
    OUTPUT_FILENAME='test.wav'
    spectrum2wav(x[0,0], sr, OUTPUT_FILENAME)

    plt_spectrum(a_content, 'Content_spectrum')
    plt_spectrum(a_style, 'Style_spectrum')
    a = np.zeros_like(a_content)
    a[:N_CHANNELS,:] = np.exp(x[0,0]) - 1
    plt_spectrum(a, 'Gen_spectrum')

if __name__ == "__main__":
    main()

Starting audio style transfer program.
   Building transfer model.
   Beginning transfer.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [38]:
from IPython.display import Audio, display
print("Starting audio style transfer program.")
a_content, a_style, N_SAMPLES, sr = preprocessing(content, style)
display(Audio(content))
display(Audio(style))
x = styleTransfer(a_content, a_style, N_SAMPLES)
    
OUTPUT_FILENAME = 'drive/MyDrive/randomCNN-voice-transfer-master/outputCnn.wav'
spectrum2wav(x[0,0], sr, OUTPUT_FILENAME)
display(Audio(OUTPUT_FILENAME))



Starting audio style transfer program.


   Building transfer model.
   Beginning transfer.
